# Multiparticle collision dynamics

## Overview

### Questions

- What is multiparticle collision dynamics (MPCD)?
- How do I setup an MPCD fluid in bulk?
- How do transport properties of the MPCD fluid depend on the collision parameters?

### Objectives

- Demonstrate how to initialize MPCD particles using a **Snapshot**.
- Explain how to configure the MPCD **Integrator** with a **StreamingMethod**
  and **CollisionMethod**.
- Demonstrate how to run a simulation and gather data from particles.

## Boilerplate code

In [1]:
import freud
import hoomd
import hoomd.mpcd
import matplotlib
import numpy
import scipy.stats

%matplotlib inline
matplotlib.style.use("ggplot")
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

## Initialization

MPCD particles can be initialized using a [hoomd.Snapshot](https://hoomd-blue.readthedocs.io/en/stable/hoomd/snapshot.html).
Let's fill a cubic box with edge length $50 \ell$ at a number density $\rho = 5/\ell^3$,
where $\ell$ is the unit of length. First, create the **Snapshot** and set the box size:

In [2]:
L = 50
density = 5
snapshot = hoomd.Snapshot()
snapshot.configuration.box = [L, L, L, 0, 0, 0]

Then, calculate the number of particles and fill the box with positions drawn
from a uniform random distribution:

In [3]:
snapshot.mpcd.N = int(density * L**3)
rng = numpy.random.default_rng(seed=42)
snapshot.mpcd.position[:] = rng.uniform(
    low=-0.5 * L, high=0.5 * L, size=(snapshot.mpcd.N, 3)
)

The MPCD particles default to having unit mass *m*. You can change it to a
different value using `Snapshot.mpcd.mass` if you want to use something else.
Here, we set it explicitly to one for completeness:

In [4]:
mass = 1
snapshot.mpcd.mass = mass

MPCD particles typically need to start with a velocity. We can draw these
velocities from the Maxwell-Boltzmann distribution consistent with
$T = 1.0\varepsilon/k_{\rm B} $, $\varepsilon$ is the unit of energy and
$k_{\rm B}$ is the Boltzmann constant. Note that it is important to subtract off
any center-of-mass motion! MPCD is momentum-conserving, so the particles will
drift if you do not.

In [5]:
kT = 1.0
velocity = rng.normal(0.0, numpy.sqrt(kT / mass), (snapshot.mpcd.N, 3))
velocity -= numpy.mean(velocity, axis=0)
snapshot.mpcd.velocity[:] = velocity

All MPCD particles also need to have a type, although nothing is currently done
with it in the code. `Snapshot.mpcd.typeid` defaults to zero, so we can just
need to put some type name into the list, say "A".

In [6]:
snapshot.mpcd.types = ["A"]

Now that the `Snapshot` is fully initialized, we can use it to create a
`Simulation`. Note that this works even though there are no regular HOOMD
particles in the snapshot.

In [7]:
simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=1)
simulation.create_state_from_snapshot(snapshot)

## Configuring the MPCD integrator

The motion of the MPCD particles is governed by alternating streaming and
collision steps. During the streaming step, the particles move according to
Newton's equations of motion. During the collision step, the particles are
assigned to a "collision" cell and stochastically exchange momentum with each
other in a cell. The time between collisions is called the collision time. These
equations of motion are implemented by the MPCD [Integrator](https://hoomd-blue.readthedocs.io/en/stable/hoomd/mpcd/integrator.html).
The amount of time covered by a streaming step or the time between collisions is
a multiple of the timestep of this integrator.

Here we will use a timestep of $1.0\tau$ where $\tau = \sqrt{m\ell^2/\varepsilon}$
is the unit of time. This timestep will correspond to our collision time because
we only have MPCD particles (see below).

In [8]:
integrator = hoomd.mpcd.Integrator(dt=1.0)
simulation.operations.integrator = integrator

We will use the SRD collision rule with collision time $\Delta t = 1.0\tau$,
collision angle $\alpha = 130^\circ$, and a thermostat to maintain constant
temperature. We attach an [SRDCollisionMethod](https://hoomd-blue.readthedocs.io/en/stable/hoomd/mpcd/collide/stochasticrotationdynamics.html)
to the `Integrator` and have a collision occur every timestep. The default
collision cell is a cube with edge length $1\ell$. Note that this collision time
is large compared to what is typically used, but we are using it for
demonstration purposes.

In [9]:
angle = 130
integrator.collision_method = hoomd.mpcd.collide.StochasticRotationDynamics(
    period=1, angle=angle, kT=kT
)

Now, we also need to setup our streaming method.
Since we are in a box with periodic boundary conditions, we can use the [Bulk](https://hoomd-blue.readthedocs.io/en/stable/hoomd/mpcd/stream/bulk.html)
streaming method to propogate solvent particles with no confining geometry.
Streaming is only performed every collision step because the particles move
with constant velocity between collisions.

In [10]:
integrator.streaming_method = hoomd.mpcd.stream.Bulk(
    period=integrator.collision_method.period
)

Although it does not affect correctness, the performance of MPCD simulations
can be sped up considerably by periodically sorting the particles. Experience
shows about every 20 collision is a reasonable frequency for doing this in HOOMD,
so we attach a sorter that does this.

In [11]:
integrator.mpcd_particle_sorter = hoomd.mpcd.tune.ParticleSorter(
    trigger=20 * integrator.collision_method.period
)

## Diffusion coefficient

### Simulation

Now that the simulation is configured, let's run a short sequence of simulations
to estimate the diffusion coefficient of the MPCD particles.

MPCD particles do not track image flags, but their motion usually becomes
diffusive pretty quickly. Hence, we can run a short simulation where we:

1. Record the initial positions of the particles at $t_0$.
2. Run up to a short time $t_1$ for the motion to become diffusive.
3. Measure the mean squared displacement $\Delta r^2(t_1)$ between $t_0$ and $t_1$.
4. Run up to a second short time $t_2$.
5. Measure the mean squared displacement $\Delta r^2(t_2)$ between $t_0$ and $t_2$.
6. Estimate the diffusion coefficient:

   $$
   D \approx \frac{\Delta r^2(t_2) - \Delta r^2(t_1)}{6(t_2 - t_1)}
   $$

7. Repeat to obtain a desired number of samples. Compute an average and estimate
   an uncertainty from the different samples.

In [12]:
num_samples = 10
wait_time = 2
sample_time = 2
D = numpy.zeros(num_samples)
for i in range(num_samples):
    # save initial position of particles
    snapshot = simulation.state.get_snapshot()
    r0 = numpy.array(snapshot.mpcd.position)

    # run forward 1 time unit and store
    simulation.run(numpy.round(wait_time / integrator.dt).astype(int))
    snapshot = simulation.state.get_snapshot()
    r1 = numpy.array(snapshot.mpcd.position)
    box = freud.box.Box.from_box(snapshot.configuration.box)
    dr = box.wrap(r1 - r0)
    msd1 = numpy.mean(numpy.sum(dr**2, axis=1))

    # run another time unit and store
    simulation.run(numpy.round(sample_time / integrator.dt).astype(int))
    snapshot = simulation.state.get_snapshot()
    r2 = numpy.array(snapshot.mpcd.position)
    box = freud.box.Box.from_box(snapshot.configuration.box)
    dr = box.wrap(r2 - r0)
    msd2 = numpy.mean(numpy.sum(dr**2, axis=1))
    
    D[i] = (msd2 - msd1) / (6 * sample_time)

D_sim = numpy.mean(D)
D_sim_err = scipy.stats.sem(D)
print(f"{D_sim:.2e} +/- {D_sim_err:.2e}")

6.43e-01 +/- 4.28e-04


### Theoretical prediction

The self-diffusion coefficient for the SRD solvent can be
[theoretically approximated](https://doi.org/10.1007/978-3-540-87706-6_1) as:

$$
D = \frac{k_{\rm B} T \Delta t}{2 m}
    \left[\frac{3M}{(1-\cos \alpha)(M - 1 + e^{-M})} - 1 \right]
$$

where *M* is the average number of particles in a collision cell. Since our
collision cells are cubes with unit length, $M = \rho \ell^3$. This contribution
comes only from the streaming step.

In [13]:
angle_rad = numpy.radians(angle)
collision_time = integrator.dt * integrator.collision_method.period
M = density
D_theory = (0.5 * kT * collision_time / mass) * (
    (3 * M) / ((1 - numpy.cos(angle_rad)) * (M - 1 + numpy.exp(-M))) - 1)
print(f"{D_theory:.2e}")

6.39e-01


*Question*: what is the percent error between the theoretical and simulated
diffusion coefficient?

## Shear viscosity

### Theoretical prediction

The shear viscosity of the SRD collision method can also be
[estimated theoretically](https://doi.org/10.1007/978-3-540-87706-6_1). It has
two contibutions: one from the streaming step and one from the collision step.
The *kinematic* viscosity $\nu = \nu_{\rm s} + \nu_{\rm c}$ can be computed
using:

$$
\begin{align}
\nu_{\rm s} &= \frac{k_{\rm B} T \Delta t}{2 m} \left[
    \frac{5 M}{(M-1+e^{-M})(2-\cos\alpha - \cos 2\alpha)} - 1\right] \\
\nu_{\rm c} &= \frac{\ell^2}{\Delta t} \left[
    \frac{M - 1 + e^{-M}}{18 M}(1 - \cos\alpha) \right]
\end{align}
$$

It has units of $\ell^2/\tau$.

In [14]:
nu_s = (0.5 * kT * collision_time / mass) * ((5 * M) / (
    (M - 1 + numpy.exp(-M)) * (2 - numpy.cos(angle_rad) - numpy.cos(2 * angle_rad))
 ) - 1)

nu_c = (1 / collision_time) * ((M - 1 + numpy.exp(-M))/(18 * M) * (1 - numpy.cos(angle_rad)))
kinematic_viscosity = nu_s + nu_c

print(f"{kinematic_viscosity:.2f}")

0.68


The corresponding *dynamic* viscosity is $\mu = m \rho \nu$, and it has units
of $\varepsilon \tau/\ell^3$.

In [15]:
dynamic_viscosity = mass * density * kinematic_viscosity
print(f"{dynamic_viscosity:.2f}")

3.40


The dynamic viscosity can be measured from simulations using a variety of
techniques, but these are bit more involved than we have time to get into right
now. We'll use one method in a later activity!

## Additional activities

1. How does the diffusion coefficient change if you decrease the collision time?
2. Does the accuracy of the theoretical prediction for the diffusion
   coefficient get better or worse if you decrease the collision time?
3. How does the diffusion coefficient depend on collision angle?
4. If you start from a different distribution of velocities, does they still
   become Maxwell-Boltzmann distributed after you run a short simulation? Does
   it matter if there is a thermostat? You can use `numpy.histogram` to check
   the distribution of velocities.